In [8]:
#importing the required library
import selenium

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [9]:
def get_template(search_term,page_no=1):
    """Generate a URL for search Term with page number"""
    
    search_term = search_term.replace(' ','+')
    template = f'https://www.amazon.in/s?k={search_term}&page={page_no}&qid=1623864210&ref=sr_pg_{page_no}'
    
    return template

In [10]:
# Function that generated URL with the search tearm
def get_url():
    Itms = ['laptops', 'Phones', 'Headphones', 'smart watches', 'Professional Cameras', 'Printers', 'scanners', 'keyboard', 'monitors', 'Home theater', 'router']
    URL = []     
    for i in Itms:
        for j in range(1,15):
            URL.append(get_template(i,j))
    return URL        

In [11]:
#the function that scraps the URL of the front page of the reviews and rattings with the search tearm.
def get_review_url(): 
    
    hreff = []
    driver = webdriver.Edge(r"C:\Users\jitsa\Downloads\edgedriver_win64 (1)\msedgedriver.exe")
    driver.maximize_window()

    URL = get_url()
    for k in URL:
        driver.get(k) #Opening with the URL template

        for l in driver.find_elements(By.XPATH,"//a[@class = 'a-link-normal']"):
            hreff.append(l.get_attribute("href"))   
    return hreff

In [12]:
href = get_review_url()
def get_review(): #the function that scraps the URL of all the reviews and rattings
    
    driver = webdriver.Edge(r"C:\Users\jitsa\Downloads\edgedriver_win64 (1)\msedgedriver.exe") #Calling the Web Driver
    driver.maximize_window()
    rattings = []
    review = []

    for i in href:
        driver.get(i) #Opening with the URL template
        
        try: #scraping the URL of the full review pages
            link = driver.find_element(By.XPATH,"//a[@data-hook='see-all-reviews-link-foot']")
            link = link.get_attribute("href")
            link= link.split('ref=cm')[0]
        except:
            pass     
        
        for i in range(1,5): #Scraping Reviews and Rattings
            l1= f'ref=cm_cr_getr_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}'
            l = link+ l1 
            
            
            driver.get(l)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('div',{'class' : 'a-section celwidget'})
        
            for item in results:
                    try:
                        rev = item.find('span', {'data-hook': "review-body"})
                        atag = item.find(['a'], class_ = "a-link-normal")
                        if rev.span.text.replace('\n','').strip() and atag['title'][0]:
                            review.append(rev.span.text.replace('\n','').strip())
                            rattings.append(atag['title'][0])
                        else:
                            break
                    except:
                        break
       
    driver.close()
    
    rating = pd.DataFrame({'Rattings':rattings, "Review": review})
    
    #saving the dataframe in csv
    rating.to_csv("Rattings1.csv",index=False) #Creating CSV
    
    return rat


Rating = get_review()
Rating

,Rattings,Review
0,1,Please don't purchase any Lenovo product.4 mon...
1,1,Please don't purchase any Lenovo product.4 mon...
2,1,Please don't purchase any Lenovo product.4 mon...
3,4,the laptop whole package is awesome. although ...
4,4,"Pros:Good specs, others have written enough ab..."
...,...,...
42819,1,I have already deposited the router to the ser...
42820,1,Pls don't buy this router the main issue with ...
42821,1,Old Netgare 1750 is doing better job than this …
42822,1,Half of the functionality of the app are paid ...


In [13]:
Rating.to_csv("Ratings_scraped.csv",index=False)# saving Data frame as CSV